In [ ]:
from ipynb.fs.full.functions_header import *
from ipynb.fs.full.functions_karrer import *

In [ ]:
# functions to calculate DC-SBM
def get_M_rs(A, g, r, s):
    '''Return m_rs, total number of edges between group r and group s, or 2x if r = s.
    Given: Adjacency matrix A and group assignments g.'''
    return(A[g == r][:, g == s].sum())

def get_M(A, g, K):
    '''Returns Matrix M, with M_rs number of edges between group r and group s, or 2x if r = s.
    Input: Adjacency matrix A and group assignments g.'''
    M = np.zeros((K, K)).astype(int)
    
    for r in range(K):
        for s in range(K):
            M[r,s] = get_M_rs(A,g,r,s)
    
    return(M)

def get_kappa(M):
    '''Compute kappa = (kappa_r), total number of ends of edges ("stubs") that emerge from vertices in group r.
    Equivalently: sum of degrees of vertices in group r.'''
    # also possible: k_degrees[g == i].sum()
    # also possible: A[g==i].sum(), i is a group
    
    return(M.sum(axis = 1))

def get_Kappa(M):
    '''Matrix version of kappa'''
    kappa = get_kappa(M)
    return(np.outer(kappa,kappa))

def get_k_group(A, g, t):
    '''Compute vector k_t = (k_it) for given group t. Input: Adajacency matrix A and group assignments g'''
    return(A[:, g == t].sum(axis = 1))

def get_k(A, g, K):
    '''Returns matrix k = (k_it), number of edges from vertex i to vertices in group t.
    Input: Adjacency matrix A, group assignments g.'''
    k_vertex_group = np.zeros((A.shape[0], K)).astype(int)
    
    for t in range(K):
        k_vertex_group[:,t] = get_k_group(A, g, t)
        
    return (k_vertex_group)

In [ ]:
# For SBM
def get_n(g,K):
    '''Calculate n = (n_r) number of vertices in group r, based on group assignments g'''
    # to account for empty clusters
    n = np.zeros(K).astype(int)
    g_order = cluster_order(g, style = 'index')
    n[g_order[:,0]] = g_order[:,1] 
    return(n)

def get_N(g,K):
    '''Matrix version of N'''
    n = get_n(g,K)
    return(np.outer(n,n))

In [ ]:
# General Info
def get_degree(A):
    '''Calculate degree sequence k = (k_i) for given adjacency matrix A. Assumes no self-edges'''
    return(A.sum(axis = 0))

def get_u(A):
    '''Calculate number of self edges u_i for vertex i.'''
    return(np.diagonal(A))

def get_m(M):
    '''Calculate total number of edges m.'''
    # same as A.sum()/2
    return(1/2*M.sum())

In [ ]:
from scipy.special import xlogy

def get_b(x):
    '''Calculate x*log(x).'''
    return(xlogy(x,x))

def get_a(x):
    '''Calculate 2x*log(x).'''
    return(2*get_b(x))

def get_b1(x, y):
    '''Calculate x*log(y).'''
    return(xlogy(x,y))

def get_a1(x, y):
    '''Calculate 2*x*log(y).'''
    return(2*get_b1(x,y))

def DCSBM_log_lik(A, g, K):
    '''Calculate Log-Likelihood for DC-SBM, eq. 7 in (Karrer, Newman 2011).'''
    M = get_M(A, g, K = K)                # M_rs number of edges between group r and group s
    Kappa = get_Kappa(M) 
    
    M1 = xlogy(M,M) - xlogy(M,Kappa)
            
    return(M1.sum())

def DCSBM_log_lik2(A, g, K):
    '''Via individual terms. Calculate Log-Likelihood for DC-SBM, eq. 7 in (Karrer, Newman 2011).'''
    M = get_M(A, g, K = K)                # M_rs number of edges between group r and group s
    kappa = get_kappa(M) 
    
    sum1 = 0
    for r in range(K):
        for s in range(K):
            if (M[r,s] == 0):
                continue
            else:
                sum1 += M[r,s]*np.log(M[r,s]/(kappa[r]*kappa[s]))
            
    return(sum1)

def SBM_log_lik(A, g, K):
    '''Calculate Log-Likelihood for SBM, eq. 6 in (Karrer, Newman 2011).'''
    M = get_M(A, g, K = K)                # M_rs number of edges between group r and group s
    N = get_N(g,K) 
    
    M1 = xlogy(M,M) - xlogy(M,N)
    return(M1.sum())

def SBM_log_lik2(A, g, K):
    '''Via individual terms. Calculate Log-Likelihood for DC-SBM, eq. 7 in (Karrer, Newman 2011).'''
    M = get_M(A, g, K = K)                # M_rs number of edges between group r and group s
    n = get_n(g,K) 
    
    sum1 = 0
    for r in range(K):
        for s in range(K):
            if (M[r,s] == 0):
                continue
            else:
                sum1 += M[r,s]*np.log(M[r,s]/(n[r]*n[s]))
            
    return(sum1)

In [ ]:
def get_change(M, kappa, k_group, k_degrees, u, K, i, r, s):
    '''Calculate change in loglikelihood (DCSBM_log_lik) when moving vertex i from its group r to group s.
    Example: i = 10; r = g[10]; s = 4
    get_change(M, kappa, k_group, k_degrees, u, K, i, r, s)'''
    if (r == s):
        return(0)
    sum1 = 0
    
    for t in range(K):
        if(t == r or t == s):
            continue
        else:
            # M[r,t] and M[s,t] after move
            x1 = M[r,t] - k_group[i,t]
            x0 = M[r,t]
            y1 = M[s,t] + k_group[i,t]
            y0 = M[s,t] 
            
            # calculate before and after
            comp_rt = get_a(x1) - get_a(x0)
            comp_st = get_a(y1) - get_a(y0)
            sum1 += comp_rt + comp_st
    
    # Calculate before and after for rs (x2), rr, ss moves
    comp_rs = get_a(M[r,s] + k_group[i,r] - k_group[i,s]) - get_a(M[r,s])
    comp_rr = get_b(M[r,r] - 2*(k_group[i,r] + u[i])) - get_b(M[r,r])
    comp_ss = get_b(M[s,s] + 2*(k_group[i,s] + u[i])) - get_b(M[s,s])
    comp_kappa_r = - get_a(kappa[r] - k_degrees[i]) + get_a(kappa[r])
    comp_kappa_s = - get_a(kappa[s] + k_degrees[i]) + get_a(kappa[s])
    
    sum1 += comp_rs + comp_rr + comp_ss + comp_kappa_r + comp_kappa_s
    
    # print("Change in rt/st:", sum1 - (comp_rs + comp_rr + comp_ss + comp_kappa_r + comp_kappa_s))
    # print("Change in rs:", comp_rs)
    # print("Change in rr:", comp_rr)
    # print("Change in ss:", comp_ss)
    # print("Change in kappa_r:", comp_kappa_r)
    # print("Change in kappa_s:", comp_kappa_s)
    # print(sum1)
    
    return(sum1)

# correct updates in SBM
def get_change_SBM(M, kappa, k_group, k_degrees, u, K, i, r, s):
    # replace kappa with n
    if (r == s):
        return(0)
    sum1 = 0    
    
    # M1 = M.copy()
    sum1 = 0
    kappa_r_new = kappa[r] - 1
    kappa_r = kappa[r]
    kappa_s_new = kappa[s] + 1
    kappa_s = kappa[s]
    
    for t in range(K):
        if(t == r or t == s):
            continue
        else:
            x1 = M[r,t] - k_group[i,t]
            x0 = M[r,t]
            y1 = M[s,t] + k_group[i,t]
            y0 = M[s,t]
            kappa_t = kappa[t]
            
            comp_rt = get_a(x1) - get_a(x0)
            comp_st = get_a(y1) - get_a(y0)
            kappa_rt = - get_a1(x1, kappa_t*kappa_r_new) + get_a1(x0, kappa_t*kappa_r)
            kappa_st = - get_a1(y1, kappa_t*kappa_s_new) + get_a1(y0, kappa_t*kappa_s)
            
            # M1[r,t] = x1
            # M1[t,r] = x1
            # M1[s,t] = y1
            # M1[t,s] = y1
            sum1 += comp_rt + comp_st + kappa_rt + kappa_st
    
    m_rs = M[r,s] + k_group[i,r] - k_group[i,s]
    comp_rs = get_a(m_rs) - get_a(M[r,s])
    kappa_rs = - get_a1(m_rs, kappa_r_new*kappa_s_new) + get_a1(M[r,s], kappa_r*kappa_s)
    
    m_rr = M[r,r] - 2*(k_group[i,r] + u[i])
    m_ss = M[s,s] + 2*(k_group[i,s] + u[i])
    
    comp_rr = get_b(m_rr) - get_b(M[r,r])
    kappa_rr = - get_b1(m_rr, kappa_r_new*kappa_r_new) + get_b1(M[r,r], kappa_r*kappa_r)
    comp_ss = get_b(m_ss) - get_b(M[s,s])
    kappa_ss = - get_b1(m_ss, kappa_s_new*kappa_s_new) + get_b1(M[s,s], kappa_s*kappa_s)
    
    sum1 += (comp_rs + comp_rr + comp_ss +
             kappa_rs + kappa_rr + kappa_ss)
    
    # M1[r,r] = m_rr
    # M1[s,s] = m_ss
    # M1[r,s] = m_rs
    # M1[s,r] = m_rs
    
    # print("Change in rt/st:", sum1)
    # print("Change in rs:", comp_rs)
    # print("Kappa_rs:", kappa_rs)
    # print("Change in rr:", comp_rr)
    # print("Kappa_rr:", kappa_rr)
    # print("Change in ss:", comp_ss)
    # print("Kappa_ss:", kappa_ss)
    # print(sum1)
    
    return(sum1)


In [ ]:
import random

def get_best_change_node(M, kappa, k_group, k_degrees, u, K, i, r):
    '''For given node i with group g[i] = r, find best community s to change to.'''
    
    # no nodes moved, so change = 0, and best group is its own node, i.e. s = g[i] = r
    max_change = 0
    max_ind_s = r
    
    for s in range(K):
        # change here for SBM
        # change1 = get_change_SBM(M, kappa, k_group, k_degrees, u, K, i, r, s)
        change1 = get_change(M, kappa, k_group, k_degrees, u, K, i, r, s)
        
        if(change1 > max_change):
            max_ind_s = s
            max_change = change1

    # print("Node", i, "from group", r, 'to group', max_ind_s, 'with change', max_change)
    
    return max_ind_s, max_change

def get_best_change(M, kappa, k_group, k_degrees, u, K, g, greed):
    '''Find best node i with corresponding community move from g[i] to s with the maximum change in log-lik.
    Parameter greed is how "greedy" the algorithm is, i.e. how many nodes per update it will consider.'''
    max_change = 0
    max_ind_i = 0
    max_ind_r = g[0]
    max_ind_s = g[0]
    
    n_vertices = len(k_degrees)

    for i in random.sample(range(n_vertices), greed):
        ind_s, change1 = get_best_change_node(M, kappa, k_group, k_degrees, u, K, i, g[i])

        if(change1 > max_change):
            max_ind_i = i
            max_ind_r = g[i]
            max_ind_s = ind_s
            max_change = change1


    # print("Best move:", "Node", max_ind_i, "from group", max_ind_r, 'to group', max_ind_s, '|',
    #      "Change:", max_change)
    
    return(max_ind_i, max_ind_r, max_ind_s, max_change)

def update_M_kappa(M, kappa, k_group, k_degrees, u, i,r,s):
    '''Update M and kappa after moving node i from g[i] = r to community s.'''
    M1 = M.copy()
    kappa1 = kappa.copy()
    k_group1 = k_group.copy()   
    
    K = k_group.shape[1]
    
    for t in range(K):
        if(t == r or t == s):
            continue
        else:
            M1[r,t] = M[r,t] - k_group[i,t]
            M1[t,r] = M1[r,t]
            M1[s,t] = M[s,t] + k_group[i,t]
            M1[t,s] = M1[s,t]
            
    M1[r,r] = M[r,r] - 2*(k_group[i,r] + u[i])
    M1[s,s] = M[s,s] + 2*(k_group[i,s] + u[i])
    M1[r,s] = M[r,s] + k_group[i,r] - k_group[i,s]
    M1[s,r] = M[r,s] + k_group[i,r] - k_group[i,s]

    kappa1[r] -= k_degrees[i]
    kappa1[s] += k_degrees[i]
    
    return(M1, kappa1)

def update_M_n(M, kappa, k_group, k_degrees, u, i,r,s):
    '''Update M and kappa after moving node i from g[i] = r to community s.
    SBM version.'''
    # replace kappa with n when using SBM
    M1 = M.copy()
    kappa1 = kappa.copy()
    k_group1 = k_group.copy()
    
    K = k_group.shape[1]
    
    for t in range(K):
        if(t == r or t == s):
            continue
        else:
            M1[r,t] = M[r,t] - k_group[i,t]
            M1[t,r] = M1[r,t]
            M1[s,t] = M[s,t] + k_group[i,t]
            M1[t,s] = M1[s,t]

    M1[r,r] = M[r,r] - 2*(k_group[i,r] + u[i])
    M1[s,s] = M[s,s] + 2*(k_group[i,s] + u[i])
    M1[r,s] = M[r,s] + k_group[i,r] - k_group[i,s]
    M1[s,r] = M[r,s] + k_group[i,r] - k_group[i,s]

    kappa1[r] -= 1
    kappa1[s] += 1
    
    return(M1, kappa1)

def update_M_kappa1(A,g,K):
    M1 = get_M(A,g,K)
    kappa1 = get_kappa(M1)
    
    return(M1, kappa1)

In [ ]:
def karrer(A, K, greed, seed):
    '''Use seed and randomise initial assignment g. The run Newman algorithm.
    Parameter greed is how "greedy" the algorithm is, i.e. how many nodes per update it will consider.'''
    # use greed = len(A) if full set
    # np.random.seed(int(time.time()))
    np.random.seed(seed)
    g = np.random.choice(a = range(K), size = A.shape[0], replace = True)
    
    g1 = karrer_run(A, g, K, greed)

    return(g1)

def karrer_run(A, g, K, greed):
    '''Run Newman algorithm for given cluster assignment.'''
    M = get_M(A, g, K)
    kappa = get_kappa(M)
    k_group = get_k(A, g, K)
    u = get_u(A)
    
    k_degrees = get_degree(A)

    ind_iter = 0
    while(True):
        if(ind_iter%50 == 0):
            print("\n")
            print("Iteration:", ind_iter)
            print("Log-likelihood:", DCSBM_log_lik(A,g,K))
            print("\n")
        
        # Step 1: find best change
        ind_i, ind_r, ind_s, change1 = get_best_change(M, kappa, k_group, k_degrees, u, K, g, greed)

        if(change1 == 0):
            print("Final Log-likelihood after run", ind_iter - 1, ":", DCSBM_log_lik(A,g,K))
            break
        else:    
            # Step 2: update M, kappa, k_group, g (k_degrees, u, K stays fixed)
            M, kappa = update_M_kappa(M, kappa, k_group, k_degrees,u, ind_i, ind_r, ind_s)
            g[ind_i] = ind_s
            k_group = get_k(A, g, K)

            ind_iter += 1
            
    return(g)

    

In [ ]:
def karrer_SBM(A, K, greed, seed):
    '''Use seed and randomise initial assignment g. The run Newman algorithm.
    Parameter greed is how "greedy" the algorithm is, i.e. how many nodes per update it will consider.'''

    #     np.random.seed(int(time.time()))
    np.random.seed(seed)
    g = np.random.choice(a = range(K), size = A.shape[0], replace = True)
    
    g1 = karrer_run_SBM(A, g, K, greed)
    
    return(g1)

def karrer_run_SBM(A, g, K, greed):
    '''Run Newman algorithm for given cluster assignment.'''
    M = get_M(A, g, K)
    n = get_n(g,K)
    k_group = get_k(A, g, K)
    u = get_u(A)
    
    k_degrees = get_degree(A)

    ind_iter = 0
    while(True):
        if(ind_iter%50 == 0):
            print("\n")
            print("Iteration:", ind_iter)
            print("Log-likelihood:", SBM_log_lik(A,g,K))
            print("\n")
        
        # Step 1: find best change
        ind_i, ind_r, ind_s, change1 = get_best_change(M, n, k_group, k_degrees, u, K, g, greed)

        if(change1 == 0):
            print("Final Log-likelihood after run", ind_iter - 1, ":", DCSBM_log_lik(A,g,K))
            break
        else:    
            # Step 2: update M, kappa, k_group, g (k_degrees, u, K stays fixed)
            M, n = update_M_n(M, n, k_group, k_degrees,u, ind_i, ind_r, ind_s)
            g[ind_i] = ind_s
            k_group = get_k(A, g, K)

            ind_iter += 1
            
    return(g)

    